In [ ]:
### This tutorial will make you acquianted with all basic DialogFlow features.

<a id='legend'></a>
In this tutorial, we will illustrate the DialogFlow functionality using the following notation.
<img src="notation.png" width="600" height="800"> 

Here are the links on the examples:
[Example 1](https://github.com/deepmipt/dialog_flow_engine/blob/dev/examples/example_1_basics.py)
[Example 2](https://github.com/deepmipt/dialog_flow_engine/blob/dev/examples/example_2_conditions.py)
[Example 3](https://github.com/deepmipt/dialog_flow_engine/blob/dev/examples/example_3_responses.py)
[Example 4](https://github.com/deepmipt/dialog_flow_engine/blob/dev/examples/example_4_transitions.py)
[Example 5](https://github.com/deepmipt/dialog_flow_engine/blob/dev/examples/example_5_global_transitions.py)
[Example 6](https://github.com/deepmipt/dialog_flow_engine/blob/dev/examples/example_6_context_serialization.py)
[Example 7](https://github.com/deepmipt/dialog_flow_engine/blob/dev/examples/example_7_pre_response_processing.py)
[Example 8](https://github.com/deepmipt/dialog_flow_engine/blob/dev/examples/example_8_misc.py)
[Example 9](https://github.com/deepmipt/dialog_flow_engine/blob/dev/example/example_9_pre_transitions_processing.py)


<a id='actor'></a>

The object that executes dialog is `Actor`. After we define `script`, we initialize `Actor` with script, start node, fallback node and default label priority.

Start node is the node where the dialog starts. Fallback node is the node where the dialog goes when there are no available transitions. 

The dialog is defined in `script`. The `script` is a flow dictionary that contains `flows` and the optional `GLOBAL` field with parameters.

Every `flow` is also a dictionary that has multiple `nodes` and the optional `LOCAL` field with parameters. It can be convenient to denote every topic by separate `flows`. 

<a id='node'></a>

`Node` is the basic element of the framework. It has the same structure as `LOCAL` and `GLOBAL`.

<img src="script_flow_local_global_node.png" width="1200" height="600"> 

Here is the content of this structure. It is described in details below. 

<img src="4_keywords.png" width="1200" height="600">

<img src="misc.png" width="300" height="600">


<a id='response'></a>

`Node` is a dictionary, where the key is the name of node, and value is the dictionary with following keys:


* `PRE_RESPONSE_PROCESSING` ( OPTIONAL) contains the dictionary of preprocessings that are applied to the response.

* `RESPONSE` contains the response the node outputs. Response can be a string or a function. 
If `RESPONSE` is a function, it must take `Context` and `Actor` as first and second argument respectively, just like here.


```
def cannot_talk_about_topic_response(ctx: Context, actor: Actor, *args, **kwargs) -> Any:
    request = ctx.last_request
    topic_pattern = re.compile(r"(.*talk about )(.*)\.")
    topic = topic_pattern.findall(request)
    topic = topic and topic[0] and topic[0][-1]
    if topic:
        return f"Sorry, I can not talk about {topic} now."
    else:
        return "Sorry, I can not talk about that now."
```


Response function can take different arguments, but in this case it must be called with these arguments in script, like here.

```
def upper_case_response(response: str):
    # wrapper for internal response function
    def cannot_talk_about_topic_response(ctx: Context, actor: Actor, *args, **kwargs) -> Any:
        return response.upper()

    return cannot_talk_about_topic_response
```

You can see examples of working with response functions in [Example 3](https://github.com/deepmipt/dialog_flow_engine/blob/dev/examples/example_3_responses.py)

<a id='response2'></a>


If `RESPONSE` is a function, and in `PRE_RESPONSE_PROCESSING` you modify response string, you must call the function in `PRE_RESPONSE_PROCESSING`,  like here:
```
def add_prefix(prefix):
    def add_prefix_processing(ctx: Context, actor: Actor, *args, **kwargs) -> Context:
        processed_node = ctx.current_node
        processed_node.response = f"{prefix}: {processed_node.response}"
        ctx.overwrite_current_node_in_processing(processed_node)
        return ctx

    return add_prefix_processing
```

and then

```
    GLOBAL: {
        PRE_RESPONSE_PROCESSING: {
            "proc_name_1": add_prefix("l1_global"),
            "proc_name_2": add_prefix("l2_global"),
        }
    },
```

<a id='transition'></a>

* `PRE_TRANSITION_PROCESSING`  ( OPTIONAL) contains the dictionary of preprocessings that are applied to the transition.

* `TRANSITIONS` contains the dictionary of transitions from this node to other nodes. In each dictionary, key is the tuple (next_flow, next_node), and value is the condition of transition to this flow and node.

<a id='selecting'></a>

Here is how the we choose the transition to execute.

<img src="choose_transitions.png" width="800" height="600"> 

<a id='misc'></a>

* `MISC` is an optional dictionary of values that can be accessed by any other functions.

<img src="misc.png" width="600" height="600"> 



You can see examples of working with `MISC` in [Example 8](https://github.com/deepmipt/dialog_flow_engine/blob/dev/examples/example_8_misc.py). 

<a id='dfe_labels'></a>

Note that in `TRANSITIONS`, function from `dff_engine.labels` can also be the keys. You can see what different functions do in this picture.

<img src="dfe_labels.png" width="800" height="600"> 

You can find examples of using these functions in [Example 4](https://github.com/deepmipt/dialog_flow_engine/blob/dev/examples/example_4_transitions.py), [Example 7](https://github.com/deepmipt/dialog_flow_engine/blob/dev/examples/example_7_pre_response_processing.py), [Example 8](https://github.com/deepmipt/dialog_flow_engine/blob/dev/examples/example_8_misc.py) and [Example 9](https://github.com/deepmipt/dialog_flow_engine/blob/dev/example/example_9_pre_transitions_processing.py)


<a id='dfe_condition'></a>

Also, for every transition, the possible conditions supported by the library is shown here.

<img src="dfe_condition.png" width="600" height="400"> 

You can also make your own condition function, but it must take `Context` and Actor as first and second argument respectively, just like the response function. Such function can be passed as a value without evaluating.
Here is the example of such function.

```
def hi_lower_case_condition(ctx: Context, actor: Actor, *args, **kwargs) -> bool:
    request = ctx.last_request
    return "hi" in request.lower()
```

You can also pass as a condition function that takes different arguments, but it must return function that takes `Context` and `Actor`.
Here is the example of such function.

```
def predetermined_condition(condition: bool):
    # wrapper for internal condition function
    def internal_condition_function(ctx: Context, actor: Actor, *args, **kwargs) -> bool:
        # It always returns `condition`.
        return condition

    return internal_condition_function
```


You can find examples of working with conditions in [Example 2](https://github.com/deepmipt/dialog_flow_engine/blob/dev/examples/example_2_conditions.py).


<a id='local_global'></a>

`LOCAL` and `GLOBAL` have the same structure as `Node`.You can see examples of using `GLOBAL` in [Example 5](https://github.com/deepmipt/dialog_flow_engine/blob/dev/examples/example_5_global_transitions.py),[Example 7](https://github.com/deepmipt/dialog_flow_engine/blob/dev/examples/example_7_pre_response_processing.py),[Example 8](https://github.com/deepmipt/dialog_flow_engine/blob/dev/examples/example_8_misc.py) and [Example 9](https://github.com/deepmipt/dialog_flow_engine/blob/dev/example/example_9_pre_transitions_processing.py)
Among these examples, `LOCAL` is used in [Example 7](https://github.com/deepmipt/dialog_flow_engine/blob/dev/examples/example_7_pre_response_processing.py) and [Example 8](https://github.com/deepmipt/dialog_flow_engine/blob/dev/examples/example_8_misc.py).


<a id='overwrite'></a>


Any variable from `PRE_TRANSITION_PROCESSING`,`PRE_RESPONSE_PROCESSING`,`TRANSITIONS` and  `MISC` from every level can be overwritten on the lower level.

<img src="overwriting_all_variables.png" width="800" height="600"> 

You can see the example of overwriting `TRANSITIONS` in [Example 5](https://github.com/deepmipt/dialog_flow_engine/blob/dev/examples/example_5_global_transitions.py), examples of overwriting `PRE_TRANSITION_PROCESSING` values in [Example 7](https://github.com/deepmipt/dialog_flow_engine/blob/dev/examples/example_7_pre_response_processing.py), examples of overwriting `MISC` values in [Example 8](https://github.com/deepmipt/dialog_flow_engine/blob/dev/examples/example_8_misc.py) and examples of overwriting `PRE_TRANSITION_PROCESSING` in [Example 9](https://github.com/deepmipt/dialog_flow_engine/blob/dev/example/example_9_pre_transitions_processing.py).


<a id='turn_handler'></a>


Turn_handler is a function that obtains bot answer for every turn. It adds a next user request, `in_request`, into the `context`. Then `Actor` processes the `context`, in function the response is logged, and the response along with the `context` is returned.

Note that  `context` can be serialized to dict or to the json string, as in [Example 6](https://github.com/deepmipt/dialog_flow_engine/blob/dev/examples/example_6_context_serialization.py).

You can see the simple example of `turn_handler` in [Example 1](https://github.com/deepmipt/dialog_flow_engine/blob/dev/examples/example_1_basics.py).


In [ ]:
from df_engine.core.keywords import GLOBAL, TRANSITIONS, RESPONSE, MISC, LOCAL
from df_engine.core.keywords import PRE_RESPONSE_PROCESSING, PRE_TRANSITIONS_PROCESSING
from df_engine.core import Context, Actor
import df_engine.conditions as cnd
import df_engine.responses
import df_engine.labels as lbl
from examples import example_1_basics
from df_engine.core.types import NodeLabel3Type
from typing import Union, Optional, Any
from numpy import random
import logging
import re

logger = logging.getLogger(__name__)

So, here is the example of a chatbot in DialogFlow Framework. We will explain every function and give references to the tutorial.

<a id='complex'></a>

This is the [condition](#dfe_condition) function, that checks if user text is castable to string.


In [ ]:

def complex_user_answer_condition(ctx: Context, actor: Actor, *args, **kwargs) -> bool:
    request = ctx.last_request
    request_is_castable = False
    try:  # check if request is castable from str
        request = eval(request)
        request_is_castable = True
    except:
        pass
    return request_is_castable

This is the example of  [transition](#transition) function, that returns tuple (name of flow, name of node,  [priority](#selecting) )


In [ ]:
def flow_node_ok_transition(ctx: Context, actor: Actor, *args, **kwargs) -> NodeLabel3Type:
    return ("flow", "node_ok", 1.0)

This is the example of [PRE_TRANSITION_PROCESSING](#transition)  .
It saves the response  to the `previous_node_response` field of `MISC`. 


In [ ]:
def save_previous_node_response_to_ctx_processing(ctx: Context, actor: Actor, prefix=None, *args, **kwargs) -> Context:
    processed_node = ctx.current_node
    ctx.misc["previous_node_response"] = processed_node.response
    return ctx

This is the example of [PRE_RESPONSE_PROCESSING](#response) .
    
It returns function that adds to the response prefix, and calls the response function if the response is callable.
    
Note that we need to overwrite the current node with the modified node, 
calling `ctx.overwrite_current_node_in_processing(processed_node)`.


In [ ]:
def add_prefix(prefix):
    def add_prefix_processing(ctx: Context, actor: Actor, *args, **kwargs) -> Context:
        processed_node = ctx.current_node
        if not callable(processed_node.response):
            processed_node.response = f"{prefix}: {processed_node.response}"
        elif callable(processed_node.response):
            processed_node.response = f"{prefix}: {processed_node.response(ctx, actor, *args, **kwargs)}"
        # overwriting the node for context 
        ctx.overwrite_current_node_in_processing(processed_node)
        return ctx

    return add_prefix_processing

This is another example of [PRE_RESPONSE_PROCESSING](#response) . 

It returns function that adds to the response content of MISC dictionary as prefix,
and calls the response function if the response is callable.

Note that we need to overwrite the current node with the modified node, 
calling `ctx.overwrite_current_node_in_processing(processed_node)`.


In [ ]:
def add_misc():
    def add_misc_processing(ctx: Context, actor: Actor, *args, **kwargs) -> Context:
        processed_node = ctx.current_node
        if not callable(processed_node.response):
            processed_node.response = f"misc: {processed_node.misc} {processed_node.response}"
        elif callable(processed_node.response):
            processed_node.response = (
                f"misc: {processed_node.misc} " f"{processed_node.response(ctx, actor, *args, **kwargs)}"
            )
        ctx.overwrite_current_node_in_processing(processed_node)
        return ctx

    return add_misc_processing

<a id='talk_about_response'></a>

This is the example of [PRE_RESPONSE_PROCESSING](#response) 

It returns function that adds to the response prefix, and calls the response function if it is callable.
    
Note that we need to overwrite the current node with the modified node, 
calling `ctx.overwrite_current_node_in_processing(processed_node)`.


In [ ]:
def high_priority_node_transition(flow_label, label):
    def transition(ctx: Context, actor: Actor, *args, **kwargs) -> NodeLabel3Type:
        return (flow_label, label, 2.0)

    return transition

This is the simple [response](#response) function. Later, it will be called just in script.


In [ ]:
def upper_case_response(response: str):
    return response.upper()

This is another example of the [response](#response) function. It does not take any argument. 

In [ ]:
def okay_response():
    return random.choice(['OKAY', 'OK'])

This is the [response](#response) function, that logs the context and returns the dictionary with previous node and last request.
Unlike previous 2 functions it does not need to be called. 

In [ ]:
def fallback_trace_response(ctx: Context, actor: Actor, *args, **kwargs) -> Any:
    logger.warning(f"ctx={ctx}")
    return str(
        {
            "previous_node": list(ctx.labels.values())[-2],
            "last_request": ctx.last_request,
        }
    )

<a id='talk_about_response'></a>

This is also a [response](#response) function. 
    
It finds whether user said "Talk about ANY_WORD" , where ANY_WORD is literally any word.

If the WORD is found, the response is "Sorry, I can not talk about WORD now. " 

Otherwise it says "Sorry, I can not talk about that now. "

In any case, it prepends length of the dialog (in number of bot utterances) to the response.

In [11]:

def talk_about_topic_response(ctx: Context, actor: Actor, *args, **kwargs) -> Any:
    request = ctx.last_request
    topic_pattern = re.compile(r"(.*talk about )(.*)\.")
    topic = topic_pattern.findall(request)
    topic = topic and topic[0] and topic[0][-1]
    if topic:
        return f"Sorry, I can not talk about {topic} now. {len(ctx.requests)}"
    else:
        return f"Sorry, I can not talk about that now. {len(ctx.requests)}"


NameError: name 'Context' is not defined

This is an example of [condition](#dfe_condition) function. It returns `True` only if the last user utterance contains "talk about. "

In [ ]:
<a id='talk about'></a>

def talk_about_topic_condition(ctx: Context, actor: Actor, *args, **kwargs) -> bool:
    request = ctx.last_request
    return "talk about" in request.lower()

This is an example of [condition](#dfe_condition) function. It returns `True` only if the last user utterance contains "no "

In [ ]:
def no_lower_case_condition(ctx: Context, actor: Actor, *args, **kwargs) -> bool:
    request = ctx.last_request
    return "no" in request.lower()

In [ ]:
# create script of dialog

script = {
    GLOBAL: {
        TRANSITIONS: {
            ("global_flow", "start_node", 1.5): cnd.exact_match("global start"),
            ("flow", "node_hi", 1.5): cnd.exact_match("global hi"),
        },
        MISC: {"var1": "global_data", "var2": "global_data", "var3": "global_data"},
        PRE_RESPONSE_PROCESSING: {
            "proc_name_1": add_prefix("l1_global"),
        },
        PRE_TRANSITIONS_PROCESSING: {"proc_tr__name_1": save_previous_node_response_to_ctx_processing},
    },
    "global_flow": {
        "start_node": {
            RESPONSE: "INITIAL NODE",
            TRANSITIONS: {
                ("flow", "node_hi"): cnd.regexp(r"base"),
                "fallback_node": cnd.true(),
            },
        },
        "fallback_node": {
            RESPONSE: "oops",
            TRANSITIONS: {
                lbl.previous(): cnd.exact_match("initial"),
                # to global flow start node
                lbl.repeat(): cnd.true()
                # global flow, fallback node
            },
        },
    },
    "flow": {
        LOCAL: {
            MISC: {
                "var2": "rewrite_by_flow",
                "var3": "rewrite_by_flow",
            },
            PRE_RESPONSE_PROCESSING: {
                "proc_name_1": add_prefix("l1_flow"),
            },
            PRE_TRANSITIONS_PROCESSING: {"proc_tr__name_2": save_previous_node_response_to_ctx_processing},
        },
        "node_hi": {
            MISC: {"var3": "rewrite_by_hi"},
            PRE_RESPONSE_PROCESSING: {
                "proc_name_1": add_prefix("l1_flow_hi"),
                "proc_name_2": add_misc(),
            },
            RESPONSE: "Hi!!!",
            TRANSITIONS: {
                ("flow", "node_complex"): complex_user_answer_condition,
                ("flow", "node_hi"): cnd.exact_match("Hi"),
                high_priority_node_transition("flow", "node_no"): no_lower_case_condition,
                ("flow", "node_topic"): talk_about_topic_condition,
                flow_node_ok_transition: cnd.all([cnd.true(), cnd.has_last_labels(flow_labels=["global_flow"])]),
            },
        },
        "node_no": {
            MISC: {"var3": "rewrite_by_NO"},
            PRE_RESPONSE_PROCESSING: {
                "proc_name_1": add_prefix("l1_flow_no"),
                "proc_name_2": add_misc(),
            },
            RESPONSE: upper_case_response("NO"),
            TRANSITIONS: {
                ("flow", "node_complex"): complex_user_answer_condition,
                ("flow", "node_hi"): cnd.regexp(r"hi"),
                ("flow", "node_topic"): talk_about_topic_condition,
            },
        },
        "node_complex": {
            MISC: {"var3": "rewrite_by_COMPLEX"},
            PRE_RESPONSE_PROCESSING: {
                "proc_name_1": add_prefix("l1_flow_complex"),
                "proc_name_2": add_misc(),
            },
            RESPONSE: "Not string detected",
            TRANSITIONS: {
                ("flow", "node_complex"): complex_user_answer_condition,
                ("flow", "node_hi"): cnd.regexp(r"hi"),
                lbl.to_fallback(0.1): cnd.true(),
            },
        },
        "node_topic": {
            MISC: {"var3": "rewrite_by_TOPIC"},
            PRE_RESPONSE_PROCESSING: {
                "proc_name_1": add_prefix("l1_flow_topic"),
                "proc_name_2": add_misc(),
            },
            RESPONSE: talk_about_topic_response,
            TRANSITIONS: {
                lbl.forward(0.5): cnd.any([cnd.regexp(r"ok"), cnd.regexp(r"o k")]),
                lbl.backward(0.5): cnd.any([cnd.regexp(r"node complex"), complex_user_answer_condition]),
            },
        },
        "node_ok": {
            MISC: {"var3": "rewrite_by_OK"},
            PRE_RESPONSE_PROCESSING: {
                "proc_name_1": add_prefix("l1_flow_ok"),
                "proc_name_2": add_misc(),
            },
            RESPONSE: okay_response(),
            TRANSITIONS: {
                ("flow", "node_complex"): complex_user_answer_condition,
                lbl.previous(): cnd.regexp(r"previous"),
            },
        },
        "fallback_node": {  # We get to this node if an error occurred while the agent was running
            RESPONSE: fallback_trace_response,
            TRANSITIONS: {
                ("flow", "node_complex"): complex_user_answer_condition,
                "node_hi": cnd.all([cnd.exact_match("Hi"), cnd.exact_match("Hello")]),
                "node_ok": cnd.exact_match("Okey"),
            },
        },
    },
}

Here we initialize `actor`.

In [ ]:
# init actor
actor = Actor(
    script,
    start_label=("global_flow", "start_node"),
    fallback_label=("global_flow", "fallback_node"),
    label_priority=1.0,
)


# handler requests

# turn_handler - a function is made for the convenience of working with an actor

Here we initialize `turn_handler`.

Note that if the argument `true_out_response` is given, it is nust be the same as outputted response.
Otherwise we get an exception.

In [ ]:
def turn_handler(
    in_request: str,
    ctx: Union[Context, str, dict],
    actor: Actor,
    true_out_response: Optional[str] = None,
):
    # Context.cast - gets an object type of [Context, str, dict] returns an object type of Context
    ctx = Context.cast(ctx)
    # Add in current context a next request of user
    ctx.add_request(in_request)
    # pass the context into actor and it returns updated context with actor response
    ctx = actor(ctx)
    #  breakpoint()
    # get last actor response from the context
    out_response = ctx.last_response
    # the next condition branching needs for testing
    if true_out_response is not None and true_out_response != out_response:
        print("request")
        print(in_request)
        print("response was")
        print(out_response)
        print("response must be")
        print(true_out_response)
        msg = f"in_request={in_request} -> true_out_response != out_response: {true_out_response} != {out_response}"
        raise Exception(msg)
    else:
        print(f"in_request={in_request} -> NODE {list(ctx.labels.values())[-1]} RESPONSE {out_response}")
    return out_response, ctx


# edit this dialog

Here we will make an example of testing dialog.

In [ ]:
TESTING_DIALOG = []

We start with the <b>start_node</b> from `global_flow` . 

Initially, values of `MISC` variables `var1`, `var2` and `var3` were equal to `global_data` as set in `GLOBAL`.

With the user request "base", condition for transition to <b>node_hi</b> from `flow` triggered.

and the condition for transition to the fallback_node from global_flow triggered.

Using [these](#transition) rules, bot made a transition to node hi from flow.

In this new flow, variables `var2` and `var3` were rewritten to rewrite_by_flow as described [here](#overwrite)

Just after that, variable `var3` was again rewritten to rewrite_by_hi on the level of <b>node_hi</b>.

In this node,  `PRE_RESPONSE_PROCESSING` from proc_name_1 was used.

By executing function add_prefix with "l1_flow_hi", "l1_flow_hi" was prepended to the `RESPONSE` from the left.

Then, `PRE_RESPONSE_PROCESSING` from proc_name_2 was used.

It executed function `add_misc()`, 
prepending to the response string "misc: {'var1': 'global_data', 'var2': 'rewrite_by_flow', 'var3': 'rewrite_by_hi'}"" 

This string contains string of all MISC values after the string "misc:"

So the final answer on base is the same as on ANSWER_1

In [ ]:
UTTERANCE_1 = "base"
ANSWER_1 =  "misc: {'var1': 'global_data', 'var2': 'rewrite_by_flow', 'var3': 'rewrite_by_hi'} " \
                "l1_flow_hi: Hi!!!"
TESTING_DIALOG.append((UTTERANCE_1, ANSWER_1))

Among the transitions in <b>node_hi</b>, only transition to <b>node_no</b> was fulfilled.

In this node, variable `var3` was rewritten and first `PRE_RESPONSE_PROCESSING` ( name `proc_name_1` ) also was rewritten. 

Then, rewritten `PRE_RESPONSE_PROCESSING` functions were executed for response just like in the previous turn.
Note that the response by itself was the result of function ```upper_case_response("NO")```

In [ ]:
UTTERANCE_2 = "no"
ANSWER_2 ="misc: {'var1': 'global_data', 'var2': 'rewrite_by_flow', 'var3': 'rewrite_by_NO'} " \
               "l1_flow_no: NO"
TESTING_DIALOG.append((UTTERANCE_2, ANSWER_2))

Among the transitions in <b>node_no</b>, conditions for transitions to <b>node_complex</b> and <b>node_hi</b> were not fulfilled.

However, condition for transition to <b>node_topic</b> was fulfilled (as utterance triggered the [function](#talk_about) )

So the response was obtained using this [function](#talk_about_response) , with a preprocessing as on the previous stages:
    `var3` variable rewritten, `misc` value and node_specific prefix prepended.

This function was given as an argument without execution ( unlike previous function) , as it takes `context` and `actor` as arguments.

In [ ]:
UTTERANCE_3 = "talk about books"
ANSWER_3 = "misc: {'var1': 'global_data', 'var2': 'rewrite_by_flow', 'var3': 'rewrite_by_TOPIC'} " \
            "l1_flow_topic: Sorry, I can not talk about that now. 3"
TESTING_DIALOG.append((UTTERANCE_3, ANSWER_3))

Among the possible transitions in <b>node_topic</b>, only transition to <b>node_ok</b> was satisfied.

This is because condition `cnd.regexp(r"ok")` was satisfied. So, transition with condition `cnd.any([cnd.regexp(r"ok"), cnd.regexp(r"o k")])` was executed.

So we went to the <b>node_ok</b> where `add_prefix` function switched to <i>l1_flow_ok</i> and variable name of `var3` changed to <i>rewrite_by_ok</i>

Then, with prepending 2 pre_response_processings, response function was executed.It randomly chooses between OK and OKAY.


In [ ]:
UTTERANCE_4 = "ok"
ANSWER_4 = "misc: {'var1': 'global_data', 'var2': 'rewrite_by_flow', 'var3': 'rewrite_by_OK'} " \
           "l1_flow_ok: OK"
TESTING_DIALOG.append((UTTERANCE_4, ANSWER_4))

Among the transitions in <b>node_ok</b>, transition with `complex_user_answer_condition` was not satisfied
as the word previous is not castable to anything other than string.

However, transition to [`lbl.previous()`](#dfe_labels) was satisfied, as the `cnd.regexp(r"previous")` was satisfied.

`lbl.previous()` redirected the user to the node which was visited by user before the <b>node_ok</b>. This is <b>node_topic</b>.

On this node, the response was processed in the same way as it was the last time we were in this node, with the exception that the dialog length is higher by 2.

UTTERANCE_5 = "previous"
ANSWER_5 =  "misc: {'var1': 'global_data', 'var2': 'rewrite_by_flow', 'var3': 'rewrite_by_TOPIC'} " \
            "l1_flow_topic: Sorry, I can not talk about that now. 5",
TESTING_DIALOG.append((UTTERANCE_5, ANSWER_5))

Among the transitions in <b>node_topic</b>,
transition with condition  ```cnd.any([cnd.regexp(r"ok"), cnd.regexp(r"o k")])``` was not satisfied 
as there were no 'ok' and 'o k' in the string.

However, condition ```cnd.any([cnd.regexp(r"node complex"), complex_user_answer_condition])``` was satisfied, 
as [complex](#complex_answer_condition) was executed to '{1:2}' and outputted True, 
just because ```'{1:2}'``` is castable to anything other than string.
    
So, despite ```cnd.regexp(r"node complex")``` was not fulffilled, condition `cnd.any` was fulfilled and we went to the <b>node_complex</b>.
    In this node, `var3` from `MICS` was rewritten, and output with two `PRE_RESPONSE_PROCESSING`s, in the same way as in previous nodes, was executed.

UTTERANCE_6 = "{1:2}"
ANSWER_6 = "misc: {'var1': 'global_data', 'var2': 'rewrite_by_flow', 'var3': 'rewrite_by_COMPLEX'} " \
           "l1_flow_complex: Not string detected",
TESTING_DIALOG.append((UTTERANCE_6, ANSWER_6))

Among the transitions in <b>node_complex</b>,
only one condition was fulfilled: condition `lbl.to_fallback(0.1)`, as it is always True
    
Hence, the next node was <b>fallback_node</b> from the global flow. The only `PRE_RESPONSE_PROCESSING` that works there is `PRE_RESPONSE_PROCESSING` on the `GLOBAL` level(as it is not [overwritten](#overwrite) by other nodes)
It added prefix `l1_global` to the `RESPONSE` "oops". So the answer is as follows.

In [ ]:
UTTERANCE_7 = "f"
ANSWER_7 = "l1_global: oops"
TESTING_DIALOG.append((UTTERANCE_7, ANSWER_7))

This is the function for testing the dialog by executing `turn_handler`.
In `turn_handler`, this function checks whether the bot outputs the same responses as we want.

The function has the functionality to cast context to string or json, as in [Example 6](https://github.com/deepmipt/dialog_flow_engine/blob/dev/examples/example_6_context_serialization.py)


In [ ]:
def run_test(mode=None):
    ctx = {}
    for in_request, true_out_response in TESTING_DIALOG:
        _, ctx = turn_handler(in_request, ctx, actor, true_out_response=true_out_response)
        if mode == "json":
            ctx = ctx.json()
            if isinstance(ctx, str):
                logging.info("context serialized to json str")
            else:
                raise Exception(f"ctx={ctx} has to be serialized to json string")
        elif mode == "dict":
            ctx = ctx.dict()
            if isinstance(ctx, dict):
                logging.info("context serialized to dict")
            else:
                raise Exception(f"ctx={ctx} has to be serialized to dict")
run_test()

Here is the function for interacting with chatbot. This function repeatedly asks user for an input and gives this input to the `turn_handler`, which logs an output.

In [ ]:
# interactive mode
def run_interactive_mode(actor):
    ctx = {}
    while True:
        in_request = input("type your answer: ")
        _, ctx = turn_handler(in_request, ctx, actor)

Here you can chat with your chatbot. Have fun!

In [ ]:
                                          
if __name__ == "__main__":
    logging.basicConfig(
        format="%(asctime)s-%(name)15s:%(lineno)3s:%(funcName)20s():%(levelname)s - %(message)s",
        level=logging.INFO,
    )
    # run_test()
    run_interactive_mode(actor)